In [5]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests

In [6]:
class HelperFunctions():
    def getYears(self,quarters): #go through each file name and find the year of filing
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    def getExcelSheetsPostDate(self,Excel): #use IncomeExcel dictionary to get all dates and only keep where date is later than year specified 
        quarters = Excel.keys()
        fileDates = self.getYears(quarters) 
        files = [file for file, date in zip(Excel, fileDates) if date>self.fromDate or (date==self.fromDate and "Year Ended" in file)]
        Excel = {file:Excel[file] for file in files}
        return Excel
        
    def reorderQuarters(self):
        quarters = self.getQuarters()
        years = self.getUniqueYears(quarters)
        cols = [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"] if q + " " + str(year) in quarters]
        self.compiledStatement = self.compiledStatement[["label"] + cols]
        
    def getQuarters(self):
        return list(self.compiledStatement.columns[1:])
    
    
    def cleanup_label(self, label):
        new_label = label.lower()
        new_label = new_label.replace("gain","loss").replace("decrease","increase")
        new_label = re.sub("\s\([\w\W]+?\)","",new_label).replace("  "," ").replace("—"," - ").strip()
        new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
        return new_label
    

In [45]:
class Income(HelperFunctions):
    def __init__(self,ticker,fromDate):
        
        self.ticker = ticker
        self.fromDate = fromDate
        self.compiledStatement = pd.DataFrame(columns=["label"])
        
        self.IncomeExcel = self.readIncomeExcel()
        self.IncomeExcel = self.getExcelSheetsPostDate(self.IncomeExcel)
        
        self.compiledStatement = self.addData()
#         self.performIncomeMath()
#         self.reorderQuarters()
        
    def readIncomeExcel(self):
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        file = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        IncomeExcel = pd.read_excel(file,sheet_name=None)
        return IncomeExcel
    
    def addData(self):
        compiledStatement = pd.DataFrame(columns=["title","label"])
        for quarter,sheet in list(self.IncomeExcel.items()):
            print(quarter)
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            sheet.label = sheet.label.apply(self.cleanup_label)
            
            new_sheet = list()
            title = sheet.loc[0,"label"]
            for i, row in sheet.iterrows():
                if pd.isnull(row[quarter]):
                    title = row["label"]
                    continue
                
                new_sheet.append([title,row["label"], row[quarter]])
            
            new_sheet = pd.DataFrame(new_sheet, columns=["title","label",quarter])
            self.i = new_sheet
            
            compiledStatement = pd.merge(compiledStatement,new_sheet,on=["title","label"],how="outer")
            
        return compiledStatement
    def performIncomeMath(self):
        quarters = self.getQuarters()
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                
                self.compiledStatement["Q4 "+year] = self.compiledStatement["Year Ended "+year] - self.compiledStatement["Q3 "+year]
                self.compiledStatement.loc[self.compiledStatement.label.str.startswith("weighted-average"),"Q4 "+year] = self.compiledStatement.loc[self.compiledStatement.label.str.startswith("weighted-average"),"Year Ended "+year]


            

In [46]:
income = Income("TSLA",2017)
income.compiledStatement

Year Ended 2017
Q1 2018
Q2 2018
Q3 2018
Year Ended 2018
Q1 2019
Q2 2019
Q3 2019
Year Ended 2019
Q1 2020
Q2 2020
Q3 2020
Year Ended 2020
Q1 2021
Q2 2021
Q3 2021
Year Ended 2021
Q1 2022
Q2 2022


,title,label,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,Year Ended 2018,Q1 2019,Q2 2019,Q3 2019,...,Q1 2020,Q2 2020,Q3 2020,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Year Ended 2021,Q1 2022,Q2 2022
0,revenues,automotive sales,8534752.0,2561881.0,3117865.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15514.0,13670
1,revenues,automotive sales,8534752.0,2561881.0,3117865.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15514.0,10153
2,revenues,automotive leasing,1106548.0,173436.0,239816.0,220461.0,883461.0,215120.0,208362.0,221.0,...,239.0,268.0,265.0,1052.0,297.0,332.0,385.0,NaN,668.0,588
3,revenues,automotive leasing,1106548.0,173436.0,239816.0,220461.0,883461.0,215120.0,208362.0,221.0,...,239.0,268.0,265.0,1052.0,297.0,332.0,385.0,NaN,668.0,368
4,revenues,total automotive revenues,9641300.0,2735317.0,3357681.0,6098766.0,18514983.0,3723861.0,5376389.0,5353.0,...,5132.0,5179.0,7611.0,27236.0,NaN,NaN,NaN,NaN,16861.0,14602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333613,revenues,weighted average shares used in computing net ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1333614,revenues,automotive revenues [member],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1333615,revenues,total automotive cost of revenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10521
1333616,revenues,energy generation and storage [member],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [42]:
income.compiledStatement

In [133]:
income.sheet

,label,Q1 2020
0,revenue,33055.00
1,cost of revenue,10406.00
2,gross margin,22649.00
3,research and development,4565.00
4,sales and marketing,4337.00
5,general and administrative,1061.00
6,operating income,12686.00
7,"other income, net",0.00
8,income before income taxes,12686.00
9,provision for income taxes,2008.00
